In [1]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from PIL import Image

CHARACTERS = ['3', 'S', 'E', 'B', 'G', 'P', 'V', 'j', 'T', 'C', '9', 'F', 'm', 
              'a', '2', 'A', 'U', 'r', 'h', 'v', 'Z', 'z', 'w', 's', '1', 'R', 
              '4', 'Y', 'l', '6', 'k', 'O', 'I', 'u', 'N', 't', 'K', 'Q', 'M', 
              'W', 'X', 'D', 'd', 'b', '8', 'p', 'g', '5', 'y', 'f', 'L', 'q', 
              'J', 'n', 'i', 'x', 'c', 'H', 'e', '7']

# Re-declare StringLookup layer with vocabulary used during training
char_to_num = layers.StringLookup(vocabulary=list(CHARACTERS),
                                  num_oov_indices=0,
                                  mask_token=None)

num_to_char = layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), 
                                  mask_token=None, 
                                  num_oov_indices=0,
                                  invert=True)

IMG_WIDTH, IMG_HEIGHT = 150, 40  
model = load_model('captcha_model.h5', custom_objects={'StringLookup': char_to_num})


def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.transpose(img, perm=[1, 0, 2])
    img = tf.expand_dims(img, axis=0) 
    return img


def decode_predictions(preds):
    output_text = []
    for i in range(preds.shape[1]):
        char_index = tf.argmax(preds[0][i])
        char = num_to_char(char_index)
        output_text.append(char)
    return tf.strings.reduce_join(output_text).numpy().decode("utf-8")


def predict(image_path):
    prediction = model.predict(preprocess_image(image_path))    
    return decode_predictions(prediction)

In [2]:
image_path = 'dataset/1a4lr.jpg'  # Replace with your image
predicted_text = predict(image_path)
print("Predicted text:", predicted_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Predicted text: 1a4lr
